<a href="https://colab.research.google.com/github/1000Roses/NLP_course/blob/main/Bert_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install transformers
!pip install underthesea

In [34]:
from underthesea import word_tokenize
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

# For Bert
import torch
from transformers import AutoModel, AutoTokenizer

# For read file
import glob
print(glob.glob("/content/drive/MyDrive/NLP/chatbotdata/*.txt"))

['/content/drive/MyDrive/NLP/chatbotdata/giải trí.txt', '/content/drive/MyDrive/NLP/chatbotdata/du lịch.txt', '/content/drive/MyDrive/NLP/chatbotdata/shoping.txt', '/content/drive/MyDrive/NLP/chatbotdata/gia đình.txt', '/content/drive/MyDrive/NLP/chatbotdata/tán gẫu.txt', '/content/drive/MyDrive/NLP/chatbotdata/các câu hỏi phức tạp.txt', '/content/drive/MyDrive/NLP/chatbotdata/đất nước.txt', '/content/drive/MyDrive/NLP/chatbotdata/nghỉ lễ.txt', '/content/drive/MyDrive/NLP/chatbotdata/học tập.txt', '/content/drive/MyDrive/NLP/chatbotdata/thông tin cá nhân.txt', '/content/drive/MyDrive/NLP/chatbotdata/sở thích.txt', '/content/drive/MyDrive/NLP/chatbotdata/trò chuyện về đi ăn.txt', '/content/drive/MyDrive/NLP/chatbotdata/robot.txt', '/content/drive/MyDrive/NLP/chatbotdata/địa chỉ.txt', '/content/drive/MyDrive/NLP/chatbotdata/người yêu.txt', '/content/drive/MyDrive/NLP/chatbotdata/tdtu.txt', '/content/drive/MyDrive/NLP/chatbotdata/bạn bè.txt', '/

In [35]:
all_path = glob.glob("/content/drive/MyDrive/NLP/chatbotdata/*.txt")

In [38]:
FRIEND_PRE_DATA = list()
VOCAB_SIZE = 0
ALL_WORDS = ['<sos>', '<eos>', ' ']

In [39]:
for path in all_path:
  with open(path) as frienddata:
    for data in frienddata:
      sent = data.split("__eou__")[:2]
      if sent[0] == ' ' or sent[1] == ' ':
        continue
      if len(sent) == 2:
        anq = list()
        for subsent in sent:
          # Remove leading, ending space in string, " để mùa hạ qua từng đêm vắng  "  -> "để mùa hạ qua từng đêm vắng"
          subsent = subsent.strip()
          anq.append(subsent)
        FRIEND_PRE_DATA.append(anq)

In [40]:
len(FRIEND_PRE_DATA)

5852

In [41]:
df = pd.DataFrame(FRIEND_PRE_DATA, columns=["Question", "Answer"])

In [42]:
questions_lower = [sent.lower() for sent in df['Question']]
df['Question'] = questions_lower

answer_lower = [sent.lower() for sent in df['Answer']]
df['Answer'] = answer_lower

In [43]:
df.head(10)

,Question,Answer
0,bạn thích hãng phim nào nhất ?,mình thích hãng phim marvel nhất
1,thể loại phim bạn thích là gì ?,"mình thích thể loại phim hành động, viễn tưởng"
2,bạn có thần tượng ca sĩ nào không ?,mình rất thích ca sĩ noo phước thịnh
3,bạn có muốn thử vận may vào việc chơi đề ?,mình không muốn cờ bạc
4,tết này bạn có muốn đi xem phim không ?,"tất nhiên rồi, mình rất muốn"
5,cuối tuần này bạn có rãnh hay không?,cuối tuần này mình rãnh
6,bạn có muốn đi xem phim vào cuối tuần này hay ...,dĩ nhiên là muốn rồi
7,mai bạn có muốn đi đá banh chung với tụi mình ...,xin lỗi mai mình có việc bận rồi
8,bạn có dự định đi chơi đâu vào cuối tuần không?,không hiện tại mình chưa có dự định gì
9,bạn cho mình hỏi bạn thích xem phim thể loại gì?,tớ thích xem phim thể loại hành động


In [44]:
question_word_segmentation = [ " ".join([ e.replace(' ','_') for e in word_tokenize(sent.lower())]) for sent in df['Question']]
df['Question_word_segmentation'] = question_word_segmentation

answer_word_segmentation = [ " ".join([ e.replace(' ','_') for e in word_tokenize(sent.lower())]) for sent in df['Answer']]
answer_word_segmentation = [ "<sos> " + sent + " <eos>" for sent in answer_word_segmentation]
df['Answer_word_segmentation'] = answer_word_segmentation

In [45]:
df.head(10)

,Question,Answer,Question_word_segmentation,Answer_word_segmentation
0,bạn thích hãng phim nào nhất ?,mình thích hãng phim marvel nhất,bạn thích hãng phim nào nhất ?,<sos> mình thích hãng phim marvel nhất <eos>
1,thể loại phim bạn thích là gì ?,"mình thích thể loại phim hành động, viễn tưởng",thể_loại phim bạn thích là gì ?,"<sos> mình thích thể_loại phim hành_động , viễ..."
2,bạn có thần tượng ca sĩ nào không ?,mình rất thích ca sĩ noo phước thịnh,bạn có thần_tượng ca_sĩ nào không ?,<sos> mình rất thích ca_sĩ noo phước thịnh <eos>
3,bạn có muốn thử vận may vào việc chơi đề ?,mình không muốn cờ bạc,bạn có muốn thử vận may vào việc chơi đề ?,<sos> mình không muốn cờ_bạc <eos>
4,tết này bạn có muốn đi xem phim không ?,"tất nhiên rồi, mình rất muốn",tết này bạn có muốn đi xem phim không ?,"<sos> tất_nhiên rồi , mình rất muốn <eos>"
5,cuối tuần này bạn có rãnh hay không?,cuối tuần này mình rãnh,cuối tuần này bạn có rãnh hay không ?,<sos> cuối tuần này mình rãnh <eos>
6,bạn có muốn đi xem phim vào cuối tuần này hay ...,dĩ nhiên là muốn rồi,bạn có muốn đi xem phim vào cuối tuần này hay ...,<sos> dĩ_nhiên là muốn rồi <eos>
7,mai bạn có muốn đi đá banh chung với tụi mình ...,xin lỗi mai mình có việc bận rồi,mai bạn có muốn đi đá banh chung với tụi_mình ...,<sos> xin_lỗi mai mình có việc bận rồi <eos>
8,bạn có dự định đi chơi đâu vào cuối tuần không?,không hiện tại mình chưa có dự định gì,bạn có dự_định đi chơi đâu vào cuối tuần không ?,<sos> không hiện_tại mình chưa có dự_định gì <...
9,bạn cho mình hỏi bạn thích xem phim thể loại gì?,tớ thích xem phim thể loại hành động,bạn cho mình hỏi bạn thích xem phim thể_loại gì ?,<sos> tớ thích xem phim thể_loại hành_động <eos>


# all word, word2index, index2word

In [46]:
for question_sent_list, answer_sent_list in zip(df['Question_word_segmentation'], df['Answer_word_segmentation']):
  for word in question_sent_list.split():
    if word not in ALL_WORDS:
      ALL_WORDS.append(word)
  for word in answer_sent_list.split():
    if word not in ALL_WORDS:
      ALL_WORDS.append(word)


In [47]:
word2index= {word: i for i, word in enumerate(ALL_WORDS)}

In [48]:
index2word= {i: word for i, word in enumerate(ALL_WORDS)}

# Bert


In [49]:
import torch
from transformers import AutoModel, AutoTokenizer

In [50]:
phobert_model = AutoModel.from_pretrained("vinai/phobert-base")

# For transformers v4.x+: 
phobert_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [51]:
# Convert word to id
question_id_each_sent = [ phobert_tokenizer.encode(line) for line in df['Question_word_segmentation']]
answer_id_each_sent = [ phobert_tokenizer.encode(line) for line in df['Answer_word_segmentation']]

# convert list to torch tensor format
question_id_each_sent = [torch.tensor(sent) for sent in question_id_each_sent]
answer_id_each_sent = [torch.tensor(sent) for sent in answer_id_each_sent]

In [18]:
answer_id_each_sent[:5][0]

tensor([    0, 63117,  3227,  1302,   884,    68,   543,   538,   266,  5276,
        27410,    67, 63117,  1358,  3603,   884,     2])

# Build bert embedding

In [52]:
max_sequence_question = max(df['Question'], key = lambda x : len(x))
max_len_question = len(max_sequence_question.split())

In [53]:
max_encoder_seq_length = max_len_question
max_decoder_seq_length = 15
num_decoder_tokens = num_encoder_tokens = 768 

In [54]:
# 3D array of shape (num_pairs, max_sentence_length_word, num_word)
encoder_input_data = np.zeros(
    (len(df['Question']), max_encoder_seq_length, num_encoder_tokens), dtype= 'float32'
)
decoder_input_data = np.zeros(
    (len(df['Question']), max_decoder_seq_length, num_decoder_tokens), dtype= 'float32'
)
decoder_target_data = np.zeros(
    (len(df['Question']), max_decoder_seq_length, len(ALL_WORDS)), dtype= 'float32'
)

In [55]:
encoder_input_data.shape, decoder_input_data.shape, decoder_target_data.shape

((5852, 93, 768), (5852, 15, 768), (5852, 15, 4768))

In [56]:
eos_vector= np.random.rand(768)

In [57]:
for i, (question_sent, answer_sent) in enumerate(zip(df['Question_word_segmentation'], df['Answer_word_segmentation'])):
  question_sent_id= torch.tensor([phobert_tokenizer.encode(question_sent)])
  answer_sent_id= torch.tensor([phobert_tokenizer.encode(answer_sent)])

  question_vector = phobert_model(question_sent_id)[0][0]
  answer_vector = phobert_model(answer_sent_id)[0][0]

  for t, word in enumerate(question_sent.split()):
    try:
      if word == '<eos>':
        encoder_input_data[i, t]= eos_vector
      else:
        encoder_input_data[i, t] = question_vector[t+1].detach().numpy()
    except:
      break

  encoder_input_data[i, t + 1:]= eos_vector

  for t, word in enumerate(answer_sent.split()):
    if t == max_decoder_seq_length:
      break
    if word == '<eos>':
      decoder_input_data[i, t]= eos_vector
    else:
      decoder_input_data[i, t] = answer_vector[t+1].detach().numpy()
    if t > 0:
        decoder_target_data[i, t - 1, word2index[word]] = 1
  decoder_input_data[i, t + 1: ]= eos_vector 
  decoder_target_data[i, t:, word2index['<eos>']] = 1 

In [58]:
soss = list()
for i in range(len(decoder_input_data)):
  soss.append(decoder_input_data[i][0])
soss = np.array(soss)
start_sos = np.mean(soss, axis= 0)

In [60]:
encoder_input_data.shape, decoder_input_data.shape, decoder_target_data.shape

((5852, 93, 768), (5852, 15, 768), (5852, 15, 4768))

# LSTM

In [61]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Bidirectional

latent_dim = 400

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, 768))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c, *_ = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, 768))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(len(ALL_WORDS), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, None, 768)]  0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, None, 768)]  0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, 400), (None, 1870400     input_5[0][0]                    
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, None, 400),  1870400     input_6[0][0]                    
                                                                 lstm_2[0][1]               

In [ ]:
batch_size = 500
epochs = 500
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics= ['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          shuffle=True,
          validation_split=0.2)

Epoch 1/500
10/10 [==============================] - 123s 12s/step - loss: 5.5888 - accuracy: 0.3901 - val_loss: 3.5743 - val_accuracy: 0.4770
Epoch 2/500
10/10 [==============================] - 115s 12s/step - loss: 3.0133 - accuracy: 0.5506 - val_loss: 3.5160 - val_accuracy: 0.4777
Epoch 3/500


In [29]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=500,
          shuffle=True,
          validation_split=0.2)

Epoch 1/500
2/2 [==============================] - 3s 1s/step - loss: 0.2417 - accuracy: 0.9500 - val_loss: 3.2230 - val_accuracy: 0.6327
Epoch 2/500
2/2 [==============================] - 2s 1s/step - loss: 0.2433 - accuracy: 0.9488 - val_loss: 3.1534 - val_accuracy: 0.6412
Epoch 3/500
2/2 [==============================] - 2s 1s/step - loss: 0.2006 - accuracy: 0.9627 - val_loss: 3.0461 - val_accuracy: 0.6352
Epoch 4/500
2/2 [==============================] - 2s 1s/step - loss: 0.1727 - accuracy: 0.9694 - val_loss: 3.0806 - val_accuracy: 0.6339
Epoch 5/500
2/2 [==============================] - 2s 1s/step - loss: 0.1616 - accuracy: 0.9744 - val_loss: 3.0877 - val_accuracy: 0.6291
Epoch 6/500
2/2 [==============================] - 2s 1s/step - loss: 0.1658 - accuracy: 0.9701 - val_loss: 3.0566 - val_accuracy: 0.6279
Epoch 7/500
2/2 [==============================] - 2s 1s/step - loss: 0.1518 - accuracy: 0.9772 - val_loss: 3.1036 - val_accuracy: 0.6364
Epoch 8/500
2/2 [=================

In [30]:
# inference setup
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [31]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0]= start_sos

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        # print(output_tokens.shape) # 1 1 112
        word_index = np.argmax(output_tokens[0, -1, :])
        word = index2word[word_index]

        # Exit condition: either hit max length
        # or find stop character.
        if (word == '<eos>' or
           len(decoded_sentence) > 50):
            stop_condition = True
            break
        decoded_sentence += word + " "


        # segment
        sent_segment=  " ".join([ e.replace(' ','_') for e in word_tokenize(decoded_sentence.lower())]) 
        # convert to id
        sent_segment_id= torch.tensor([phobert_tokenizer.encode(sent_segment)])
        # get last embedding
        last_word_vector = phobert_model(question_sent_id)[0][0][-1].detach().numpy()
        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0] = last_word_vector

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [32]:
for i in range(25):
  print(decode_sequence(np.array([encoder_input_data[i]])))

mình thích phim 
mình thích là 
mình rất rất 
mình không muốn muốn 
tất_nhiên rồi là là 
cuối tuần này này 
dĩ_nhiên là là là muốn 
xin_lỗi mai mình có 
không hiện_tại hiện_tại . chơi 
tớ thích xem phim là là 
tớ không hay chơi chơi chơi chơi chơi 
tớ thích xem là là là là là là là là là chơi chơi chơi 
tớ thường đi chơi chơi là là là là là là bạn chơi chơi 
tớ không không chơi 
tớ thích ăn chơi chơi chơi chơi chơi 
phim cgv cgv 
ừ là là là 
ok , coi . . . . . 
tùy cậu cũng cũng . 
ok cậu cậu . . 
ở cgv cgv 
phim cgv cgv 
ừ là là là 
ok , coi . . . . . 
tùy cậu cũng cũng . 


# Khi ma lam cau truy van, nho lam word segmentation